In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Transformations in Code
todo

## Xform Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D96 Xform Large.jpg" style="width: 600px; display: inline;">

In [ ]:
"""
Xform Initialization
An Xform object is initialized with values configured to match those of an 
identity matrix.
"""

class Xform(object):

    def __init__(self):
        self._m = [0.0]*16
        self.c11, self.c22, self.c33, self.c44, = 1.0, 1.0, 1.0, 1.0

In [ ]:
"""
Transformation Management
Here, a method is defined that attempts to apply a spatial transformation as 
described by this Xform matrix to any geometric object given. An appropriate 
mechanism for doing so is applied on a case-by-case basis, with constituent 
Points and Vecs transformed separately, stripping away the translation portion 
of the matrix where needed. If no appropriate means is defined, an error is 
raised.
"""
def transform(self,other):
    
    # if other is a Point
    if isinstance(other, Point): 
        # apply the transformation and return a new Point
        tup = self._xform_tuple(other.to_tuple())
        pt = Point(tup[0],tup[1],tup[2])
        return pt
    
    # if other is a Vec
    if isinstance(other, Vec):
        # apply the transformation and return a new Vec
        tup = self._xform_tuple(other.to_tuple())
        vec = Vec(tup[0],tup[1],tup[2])
        return vec     
    
    # if other is a LinearEntity
    if isinstance(other, LinearEntity):
        # apply the transformation to the LinearEntity start point
        other._pt = other._pt*self
        xf = self.strip_translation()
        # apply the transformation to the LinearEntity vector
        other._vec = other._vec*xf
        # assemble and return a new LinearEntity
        return other
    
    # if other is a CS
    if isinstance(other, CS):
        cs = other
        # apply the transformation to the CS origin
        tup = self._xform_tuple(cs.origin.to_tuple())
        origin = Point(tup[0],tup[1],tup[2])
        xf = self.strip_translation()
        # apply the transformation to each of the CS axis Vecs
        tup = xf._xform_tuple(cs.x_axis.to_tuple())
        x_axis = Vec(tup[0],tup[1],tup[2])
        tup = xf._xform_tuple(cs.y_axis.to_tuple())
        y_axis = Vec(tup[0],tup[1],tup[2])
        # assemble and return a new CS
        ret = CS(origin, x_axis, y_axis)
        return ret
    
    # if other is a Circle
    if isinstance(other, Circle):
        # apply the transformation as if this Circle was a Plane
        pln = other.plane * self
        cir = Circle(pln,other.rad)
        # assemble and return a new Circle
        return cir 
    
    # if other is a Plane
    if isinstance(other, Plane):
        pln = other
        # apply the transformation to the Plane origin
        tup = self._xform_tuple(pln.origin.to_tuple())
        origin = Point(tup[0],tup[1],tup[2])
        xf = self.strip_translation()
        # apply the transformation to the Plane normal Vec
        tup = xf._xform_tuple(pln.normal.to_tuple())
        normal = Vec(tup[0],tup[1],tup[2]).normalized()
        # assemble and return a new Plane
        pln = Plane(origin, normal)
        pln.copy_props(other)
        return pln

    # if no transformation is defined for this object, raise an error.
    raise NotImplementedError("I'm sorry. I can't transform that.")
    

In [ ]:
"""
A Rotation of a Segment
Following an earlier example that defined a matrix for rotating vectors, a 
given Segment seg is rotated by 90 degrees about the origin.
"""
xf = Xform()
xf.c11 = 0
xf.c12 = -1
xf.c21 = 1
xf.c22 = 0

seg *= xf

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P17.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Variable Rotation of Segments
Here, a function is defined that rotates a given termination Point of a Segment 
a variable amount as determined by its distance to an attractor Point. For this 
purpose, a single Xform object is continually altered and applied to each Point. Two given Intervals, ival_dist and ival_spin, control the amount of rotation.
"""
xf = Xform()
def spin(pt):
    # calculate an angle of rotation
    ang = Interval.remap(pt.dist(attr_pt),ival_dist,ival_spin)
    sint, cost = sin(ang), cos(ang)
    # set the components of the matrix to the desired rotation
    xf.c11, xf.c12, xf.c21, xf.c22 = cost, -sint, sint, cost
    return pt * xf

segs = [Segment(spin(seg.spt),spin(seg.ept)) for seg in segs]

In [ ]:
"""
Combining Transformations
Xform objects may be combined before being applied to a geometric object. 
Here, four transformations are defined by calling on static methods of the 
Xform class, and then combined before being recursively applied to a given 
collection of Segments. Note that the order of combination plays a significant 
role in the nature of the result.
"""
xf_rot = Xform.rotation(angle = radians(15))
xf_trn = Xform.translation(Vec(0.50, 0.33))
xf_scl = Xform.scale(1.02)
xf_mir = Xform.mirror("world_yz")

xf_sum = xf_trn * xf_rot * xf_scl

# figure is a given collection of Segments
figs = [figure]
for i in range(count):
    figs.append([xf_sum*seg for seg in figs[-1]])

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P18.jpg" style="width: 200px; display: inline;">

## A Library of Spatial Transformations

### Change of Basis

\begin{align}
     {\large M_{\vec{s_{1}},\vec{s_{2}},\vec{s_{3}} \to \vec{e_{1}},\vec{e_{2}},\vec{e_{3}}} } = 
    \begin{bmatrix}
        s_{1x} & s_{2x} & s_{3x} & 0 \\
        s_{1y} & s_{2y} & s_{3y} & 0 \\
        s_{1z} & s_{2z} & s_{3z} & 0 \\  
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
\end{align}

\begin{align}
     {\large M_{\vec{e_{1}},\vec{e_{2}},\vec{e_{3}} \to \vec{t_{1}},\vec{t_{2}},\vec{t_{3}} } } = 
    \begin{bmatrix}
        t_{1x} & t_{1x} & t_{1x} & 0 \\
        t_{2y} & t_{2y} & t_{2y} & 0 \\
        t_{3z} & t_{3z} & t_{3z} & 0 \\  
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
\end{align}

\begin{align}
    \begin{bmatrix}
        1 & 0 & 0 & o_{tx} \\
        0 & 1 & 0 & o_{ty} \\
        0 & 0 & 1 & o_{tz} \\    
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
     \times
    {\large M_{\vec{e_{1}},\vec{e_{2}},\vec{e_{3}} \to \vec{t_{1}},\vec{t_{2}},\vec{t_{3}} } } 
     \times
     {\large M_{\vec{s_{1}},\vec{s_{2}},\vec{s_{3}} \to \vec{e_{1}},\vec{e_{2}},\vec{e_{3}}} }
     \times
    \begin{bmatrix}
        1 & 0 & 0 & -o_{sx} \\
        0 & 1 & 0 & -o_{sy} \\
        0 & 0 & 1 & -o_{sz} \\    
        0 & 0 & 0 & 1 \\
    \end{bmatrix}     
\end{align}

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P27.jpg" style="width: 200px; display: inline;">